In [1]:
import os
import sys
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
from tensorflow import feature_column as fc
plt.rcParams["font.family"] = 'NanumBarunGothic'
TENSORBOARD_BINARY = '/home/hoondori/anaconda3/envs/ai/bin/tensorboard'
os.environ['TENSORBOARD_BINARY'] =  TENSORBOARD_BINARY
%load_ext tensorboard

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 텐서플로가 첫 번째 GPU만 사용하도록 제한
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
        print(e)


# TF2 로 Mobilenet v2 자체 구현 및 성능 측정 

* 참고 자료 
  * https://github.com/monatis/mobilenetv2-tf2/blob/master/mobilenetv2.py


# Mobilenet v2 구현 

*  depthwise separate conv
  * 3x3 depth conv -> 1x1 conv
* inverted residual block 
  * narrow => wide => narrow
* relu6 사용

In [6]:
#### Necessary Imports for Neural Net 

import tensorflow as tf

# Define ReLU6 activation
relu6 = tf.keras.layers.ReLU(6.)



# conv2d+Batch+Relu6 묶음
def _conv_block(inputs, filters, kernel, strides):
    
    x = tf.keras.layers.Conv2D(filters, kernel, padding='same', strides=strides)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU(6.)(x)
    
    return x
    
# basic bottleneck structure
#  - 1x1 t 배로 channel 불리기 
#  - 3x3 DepthwiseConv  (channel 수 유지)
#  - 1x1 target_channel
def _bottleneck(inputs, filters, kernel, t, strides, r=False):
    
    tchannel = inputs.shape[-1] * t
    
    x = _conv_block(inputs, tchannel, (1,1), (1,1))
    
    x = tf.keras.layers.DepthwiseConv2D(
        kernel, strides=strides, depth_multiplier=1, padding='same')(x)
    
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU(6.)(x)
    
    x = tf.keras.layers.Conv2D(filters, (1,1), strides=(1,1), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU(6.)(x)
    
    if r:
        x = tf.keras.layers.add([x, inputs])
        
    return x

def _inverted_residual_block(inputs, filters, kernel, t, strides, n):
    x = _bottleneck(inputs, filters, kernel, t, strides)
    
    for i in range(1, n):
        x = _bottleneck(x, filters, kernel, t, 1, True)

    return x

def MobileNetV2(input_shape, k):
    
    inputs = tf.keras.layers.Input(shape=input_shape, name='input')
    x = _conv_block(inputs, 32, (3, 3), strides=(2, 2))
    
    x = _inverted_residual_block(x, 16, (3, 3), t=1, strides=1, n=1)
    x = _inverted_residual_block(x, 24, (3, 3), t=6, strides=2, n=2)
    x = _inverted_residual_block(x, 32, (3, 3), t=6, strides=2, n=3)
    x = _inverted_residual_block(x, 64, (3, 3), t=6, strides=2, n=4)
    x = _inverted_residual_block(x, 96, (3, 3), t=6, strides=1, n=3)
    x = _inverted_residual_block(x, 160, (3, 3), t=6, strides=2, n=3)
    x = _inverted_residual_block(x, 320, (3, 3), t=6, strides=1, n=1)
   
    x = _conv_block(x, 1280, (1,1), strides=(1,1))
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Reshape((1,1,1280))(x)
    x = tf.keras.layers.Dropout(0.3, name='Dropout')(x)
    x = tf.keras.layers.Conv2D(k, (1,1), (1,1), padding='same')(x)  # target class
    x = tf.keras.layers.Activation('softmax', name='final_activation')(x)
    output = tf.keras.layers.Reshape((k,), name='output')(x)
    model = tf.keras.models.Model(inputs, output)
    #model.summary()

    return model


In [7]:
# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

#constant
BATCH_SIZE = 128
EPOCHS = 20
CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = tf.keras.optimizers.RMSprop()

log_dir = '/tmp/logs/mobilenetv2'

# data: shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
# normalize
X_train, X_test = X_train / 255.0, X_test / 255.0
# convert to categorical
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, CLASSES)

model = MobileNetV2(input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS), k=CLASSES)
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)
# use TensorBoard
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir=log_dir)
]

# train
model.compile(loss='categorical_crossentropy', optimizer=OPTIM,
    metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=BATCH_SIZE,
    epochs=EPOCHS, validation_split=VALIDATION_SPLIT, 
    verbose=VERBOSE, callbacks=callbacks) 
score = model.evaluate(X_test, y_test,
                     batch_size=BATCH_SIZE, verbose=VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])


Epoch 1/20
313/313 [==============================] - 8s 26ms/step - loss: 2.4044 - accuracy: 0.1377 - val_loss: 2.3328 - val_accuracy: 0.1025
Epoch 2/20
313/313 [==============================] - 7s 22ms/step - loss: 2.1090 - accuracy: 0.2326 - val_loss: 2.1169 - val_accuracy: 0.1930
Epoch 3/20
313/313 [==============================] - 7s 22ms/step - loss: 1.8510 - accuracy: 0.3100 - val_loss: 2.7155 - val_accuracy: 0.2545
Epoch 4/20
313/313 [==============================] - 7s 21ms/step - loss: 1.6652 - accuracy: 0.3883 - val_loss: 2.4511 - val_accuracy: 0.2413
Epoch 5/20
313/313 [==============================] - 7s 22ms/step - loss: 1.5039 - accuracy: 0.4543 - val_loss: 1.8716 - val_accuracy: 0.3992
Epoch 6/20
313/313 [==============================] - 7s 22ms/step - loss: 1.4069 - accuracy: 0.4979 - val_loss: 1.8296 - val_accuracy: 0.4402
Epoch 7/20
313/313 [==============================] - 7s 21ms/step - loss: 1.3589 - accuracy: 0.5244 - val_loss: 1.4752 - val_accuracy: 0.4769